In [ ]:
cd CIFAR-10/ResNet-18

In [5]:
import numpy as np, joblib
import sys, os, random
import matplotlib.pyplot as plt
import pickle, gzip
from tqdm import tqdm,tqdm_notebook
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

In [6]:
batch_size = 128
seed_num = 1000

# For reproducibility when you run the file with .py
torch.cuda.is_available()
torch.manual_seed(seed_num)
torch.cuda.manual_seed(seed_num)
np.random.seed(seed_num)
random.seed(seed_num)
torch.backends.cudnn.benchmark = True

torch.backends.cudnn.deterministic =True

In [7]:
torch.manual_seed(seed_num)
torch.cuda.manual_seed(seed_num)
np.random.seed(seed_num)
random.seed(seed_num)

# Normalization
train_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))]) 
test_transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

# Splitting the training and test datasets
train_data = datasets.CIFAR10(os.getcwd(), train=True,
                              download=True, transform=train_transform)
test_data = datasets.CIFAR10(os.getcwd(), train=False,
                             download=True, transform=test_transform)

Files already downloaded and verified
Files already downloaded and verified


In [8]:
# Split the training set indices into training and validation set indices using 80:20 ratio
np.random.seed(seed_num)
len_trainset = len(train_data)
index_list = list(range(len_trainset))
np.random.shuffle(index_list)
split_index = 40000
train_indices, valid_indices =  index_list[:split_index], index_list[split_index:]

# Creating Samplers for training and validation set using the indices
np.random.seed(seed_num)
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(valid_indices)

torch.manual_seed(seed_num)

train_iterator = DataLoader(train_data, batch_size=batch_size, sampler=train_sampler)
val_iterator = DataLoader(train_data, batch_size=batch_size, sampler=valid_sampler)
test_iterator = DataLoader(test_data, batch_size=batch_size, shuffle=True)


In [9]:
import torchvision.models as models

In [10]:
# ResNet-18 model

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        base = models.resnet18(pretrained=True)
        self.base = nn.Sequential(*list(base.children())[:-1])
        in_features = base.fc.in_features
        self.drop = nn.Dropout()
        self.final = nn.Linear(in_features,10)
    
    def forward(self,x):
        x = self.base(x)
        x = self.drop(x.view(-1,self.final.in_features))
        return self.final(x)
    
model = Model().cuda()

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [11]:
# Loading the weights of ternary model 
model = torch.load("Main_ResNet18_cifar10_Quant.pt").cuda()
print("Loading weights done !")

# Summary
print("Model:\n",model)

Loading weights done !
Model:
 Model(
  (base): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True,

In [12]:
# Layer names
layer_name = [n for n, p in model.named_parameters()]
print(layer_name)

['base.0.weight', 'base.1.bias', 'base.1.weight', 'base.4.0.conv1.weight', 'base.4.0.bn1.bias', 'base.4.0.bn1.weight', 'base.4.0.conv2.weight', 'base.4.0.bn2.bias', 'base.4.0.bn2.weight', 'base.4.1.conv1.weight', 'base.4.1.bn1.bias', 'base.4.1.bn1.weight', 'base.4.1.conv2.weight', 'base.4.1.bn2.bias', 'base.4.1.bn2.weight', 'base.5.0.conv1.weight', 'base.5.0.bn1.bias', 'base.5.0.bn1.weight', 'base.5.0.conv2.weight', 'base.5.0.bn2.bias', 'base.5.0.bn2.weight', 'base.5.0.downsample.0.weight', 'base.5.0.downsample.1.bias', 'base.5.0.downsample.1.weight', 'base.5.1.conv1.weight', 'base.5.1.bn1.bias', 'base.5.1.bn1.weight', 'base.5.1.conv2.weight', 'base.5.1.bn2.bias', 'base.5.1.bn2.weight', 'base.6.0.conv1.weight', 'base.6.0.bn1.bias', 'base.6.0.bn1.weight', 'base.6.0.conv2.weight', 'base.6.0.bn2.bias', 'base.6.0.bn2.weight', 'base.6.0.downsample.0.weight', 'base.6.0.downsample.1.bias', 'base.6.0.downsample.1.weight', 'base.6.1.conv1.weight', 'base.6.1.bn1.bias', 'base.6.1.bn1.weight', 'ba

In [13]:
# Total number of ternary weights (+w, -w)
totalParams = 0
for i in layer_name:
  print(i,(model.state_dict()[i] !=0).sum())
  totalParams +=  (model.state_dict()[i] !=0).sum()
    
print("Total Parameters:",totalParams, '\n')

base.0.weight tensor(1599, device='cuda:0')
base.1.bias tensor(52, device='cuda:0')
base.1.weight tensor(52, device='cuda:0')
base.4.0.conv1.weight tensor(2397, device='cuda:0')
base.4.0.bn1.bias tensor(52, device='cuda:0')
base.4.0.bn1.weight tensor(52, device='cuda:0')
base.4.0.conv2.weight tensor(2456, device='cuda:0')
base.4.0.bn2.bias tensor(53, device='cuda:0')
base.4.0.bn2.weight tensor(53, device='cuda:0')
base.4.1.conv1.weight tensor(2401, device='cuda:0')
base.4.1.bn1.bias tensor(52, device='cuda:0')
base.4.1.bn1.weight tensor(52, device='cuda:0')
base.4.1.conv2.weight tensor(2619, device='cuda:0')
base.4.1.bn2.bias tensor(53, device='cuda:0')
base.4.1.bn2.weight tensor(52, device='cuda:0')
base.5.0.conv1.weight tensor(1545, device='cuda:0')
base.5.0.bn1.bias tensor(104, device='cuda:0')
base.5.0.bn1.weight tensor(104, device='cuda:0')
base.5.0.conv2.weight tensor(1737, device='cuda:0')
base.5.0.bn2.bias tensor(107, device='cuda:0')
base.5.0.bn2.weight tensor(105, device='cud

In [14]:
# Model's performance on test set

correct_count, all_count = 0, 0
model.eval()
for images,labels in test_iterator:
      for image,label in zip(images,labels):

        if torch.cuda.is_available():
            img = image.cuda()
            lab = label.cuda()
            img = img[None,].type('torch.cuda.FloatTensor')

        with torch.no_grad():
            output_ = model(img) 

        pred_label = output_.argmax()

        if(pred_label.item()==lab.item()):
          correct_count += 1
        all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Test Accuracy =", (correct_count/(all_count)))


Number Of Images Tested = 10000

Model Test Accuracy = 0.544


In [15]:
# For each layer, model's ternary weights
state_dict = model.state_dict()

layer_distinct_weights = {}

for i in layer_name:
  imd = torch.unique(model.state_dict()[i])
  print(i+ ' hidden layer dimension', model.state_dict()[i].shape)
  print("Unique values of weight in "+ i+ " th hidden layer : ", imd)
  layer_distinct_weights[i] = imd.cpu().numpy().tolist()
  print()


base.0.weight hidden layer dimension torch.Size([64, 3, 7, 7])
Unique values of weight in base.0.weight th hidden layer :  tensor([-0.9980,  0.0000,  0.9980], device='cuda:0')

base.1.bias hidden layer dimension torch.Size([64])
Unique values of weight in base.1.bias th hidden layer :  tensor([-1.0023,  0.0000,  1.0049], device='cuda:0')

base.1.weight hidden layer dimension torch.Size([64])
Unique values of weight in base.1.weight th hidden layer :  tensor([0.0000, 1.0063], device='cuda:0')

base.4.0.conv1.weight hidden layer dimension torch.Size([64, 64, 3, 3])
Unique values of weight in base.4.0.conv1.weight th hidden layer :  tensor([-0.9993,  0.0000,  0.9993], device='cuda:0')

base.4.0.bn1.bias hidden layer dimension torch.Size([64])
Unique values of weight in base.4.0.bn1.bias th hidden layer :  tensor([-1.0043,  0.0000,  1.0005], device='cuda:0')

base.4.0.bn1.weight hidden layer dimension torch.Size([64])
Unique values of weight in base.4.0.bn1.weight th hidden layer :  tensor

In [16]:
img_0 = []; var_0 = 0
img_1 = []; var_1 = 0
img_2 = []; var_2 = 0
img_3 = []; var_3 = 0
img_4 = []; var_4 = 0
img_5 = []; var_5 = 0
img_6 = []; var_6 = 0
img_7 = []; var_7 = 0
img_8 = []; var_8 = 0
img_9 = []; var_9 = 0

In [17]:
correct_count, all_count = 0, 0
np.random.seed(seed_num)

num = 5000

model.eval()
for images,labels in train_iterator:
      for image,label in zip(images,labels):

        if label == 0:
          if var_0 == num:
            continue
          img_0.append(image.numpy())
          var_0 +=1

        elif label == 1:
          if var_1 == num:
            continue
          img_1.append(image.numpy())
          var_1 +=1

        elif label == 2:
          if var_2 == num:
            continue
          img_2.append(image.numpy())
          var_2 +=1

        elif label == 3:
          if var_3 == num:
            continue
          img_3.append(image.numpy())
          var_3 +=1

        elif label == 4:
          if var_4 == num:
            continue
          img_4.append(image.numpy())
          var_4 +=1

        elif label == 5:
          if var_5 == num:
            continue
          img_5.append(image.numpy())
          var_5 +=1

        elif label == 6:
          if var_6 == num:
            continue
          img_6.append(image.numpy())
          var_6 +=1

        elif label == 7:
          if var_7 == num:
            continue
          img_7.append(image.numpy())
          var_7 +=1

        elif label == 8:
          if var_8 == num:
            continue
          img_8.append(image.numpy())
          var_8 +=1

        elif label == 9:
          if var_9 == num:
            continue
          img_9.append(image.numpy())
          var_9 +=1

        if torch.cuda.is_available():
            img = image.cuda()
            lab = label.cuda()
            img = img[None,].type('torch.cuda.FloatTensor')

        with torch.no_grad():
            output_ = model(img) 

        pred_label = output_.argmax()

        if(pred_label.item()==lab.item()):
          correct_count += 1
        all_count += 1

print("Number Of Images Tested =", all_count)
print("\nModel Train Accuracy =", (correct_count/(all_count)))

Number Of Images Tested = 40000

Model Train Accuracy = 0.5474


In [21]:
def getTemplates(imgs, cls):
  np.random.seed(seed_num)
  corr_images2_1 = []
  for img in imgs:
    for angle in range(0,360,3):
        transform_st = transforms.Compose([transforms.ToTensor(), transforms.RandomRotation(angle), \
                                              transforms.RandomHorizontalFlip(),]) 
        corr_images2_1.append(transform_st(img).permute(1,2,0).numpy())
      
        transform_st1 = transforms.Compose([transforms.ToTensor(), transforms.RandomRotation(angle),]) 
        corr_images2_1.append(transform_st1(img).permute(1,2,0).numpy())
      
        transform_st2 = transforms.Compose([transforms.ToTensor(),transforms.RandomHorizontalFlip(0.25), ]) 
        corr_images2_1.append(transform_st2(img).permute(1,2,0).numpy())

        transform_st3 = transforms.Compose([transforms.ToTensor(),transforms.RandomHorizontalFlip(0.65), transforms.RandomAffine(angle)]) 
        corr_images2_1.append(transform_st3(img).permute(1,2,0).numpy())

        transform_st4 = transforms.Compose([transforms.ToTensor(),transforms.RandomRotation(angle), transforms.RandomAffine(angle)]) 
        corr_images2_1.append(transform_st4(img).permute(1,2,0).numpy())

        transform_st5 = transforms.Compose([transforms.ToTensor(),transforms.RandomRotation(angle), transforms.RandomHorizontalFlip(0.75),
                                            transforms.RandomAffine(angle)]) 
        corr_images2_1.append(transform_st5(img).permute(1,2,0).numpy())

        transform_st6 = transforms.Compose([transforms.ToTensor(),transforms.RandomRotation(angle), transforms.RandomVerticalFlip(0.75),
                                            transforms.RandomAffine(angle)]) 
        corr_images2_1.append(transform_st6(img).permute(1,2,0).numpy())

        transform_st7 = transforms.Compose([transforms.ToTensor(), transforms.RandomVerticalFlip(0.75),
                                            transforms.RandomAffine(angle)]) 
        corr_images2_1.append(transform_st7(img).permute(1,2,0).numpy())

        transform_st8 = transforms.Compose([transforms.ToTensor(), transforms.RandomVerticalFlip(0.75),]) 
        corr_images2_1.append(transform_st8(img).permute(1,2,0).numpy())

        transform_st9 = transforms.Compose([transforms.ToTensor(), transforms.RandomRotation(angle), \
                                              transforms.RandomVerticalFlip(0.25),]) 
        corr_images2_1.append(transform_st9(img).permute(1,2,0).numpy())

        transform_st10 = transforms.Compose([transforms.ToTensor(), transforms.RandomHorizontalFlip(0.75), transforms.RandomRotation(angle), \
                                              transforms.RandomVerticalFlip(0.75),]) 
        corr_images2_1.append(transform_st10(img).permute(1,2,0).numpy())

        

  corr_images2_1 = np.array(corr_images2_1)[np.random.permutation(len(corr_images2_1))]  

  corr_images2_1 = torch.tensor(corr_images2_1)

  corr_label = []

  model.eval()
  for img in corr_images2_1:
    img = img[None,].type('torch.cuda.FloatTensor')
    with torch.no_grad():
      pred = model(img).argmax()
    corr_label.append(pred.item())

  idx = np.where(np.array(corr_label) == cls)

  return np.unique(corr_images2_1[idx], axis=0)[:1000]
  

In [22]:
corr_images_ = []
for cls, im in enumerate([img_0, img_1, img_2, img_3, img_4, img_5, img_6, img_7, img_8, img_9]):
  corr_images_.append(getTemplates(np.array(im)[np.random.permutation(len(im))[:100]], cls))

In [23]:
corr_images = np.vstack(corr_images_)
corr_images.shape

(9729, 3, 32, 32)

In [24]:
corr_images = torch.tensor(corr_images).type(torch.FloatTensor).to('cuda')

corr_label = []
check_label = []

model.eval()
for img in corr_images:
  img = img[None,].type('torch.cuda.FloatTensor')
  with torch.no_grad():
    pred = model(img).argmax()
  corr_label.append(pred)
  check_label.append(pred.item())

digit,count = np.unique(check_label, return_counts=True)

# Model's prediction on pseudorandom images
print("\nUnique digits : ",digit)

print("\n counts : ",count)


Unique digits :  [0 1 2 3 4 5 6 7 8 9]

 counts :  [1000 1000 1000 1000  780 1000 1000  949 1000 1000]


In [25]:
pos_neg_layerwise = {}

for i in layer_name:
  b = layer_distinct_weights[i]

  if len(b) == 3:
    neg, zero, pos = b
    pos_neg_layerwise[i] = {}
    pos_neg_layerwise[i]['pos'] = pos
    pos_neg_layerwise[i]['neg'] = neg
    
  elif len(b) == 2:
    un1, un2 = b

    if un2 == 0:
      neg, zero = b
      pos_neg_layerwise[i] = {}
      pos_neg_layerwise[i]['neg'] = neg
      
    elif un1 == 0:
      zero, pos = b
      pos_neg_layerwise[i] = {}
      pos_neg_layerwise[i]['pos'] = pos

    else:
      neg, pos = b
      pos_neg_layerwise[i] = {}
      pos_neg_layerwise[i]['pos'] = pos
      pos_neg_layerwise[i]['neg'] = neg

  else:
     un = b[0] 

     if un > 0:
       pos_neg_layerwise[i] = {}
       pos_neg_layerwise[i]['pos'] = un

     elif un < 0:
       pos_neg_layerwise[i] = {}
       pos_neg_layerwise[i]['neg'] = un

     else:
       pos_neg_layerwise[i] = {}
       pos_neg_layerwise[i]['zero'] = 0


In [26]:
layer_distinct_weights

{'base.0.weight': [-0.997970700263977, 0.0, 0.997970700263977],
 'base.1.bias': [-1.002286434173584, 0.0, 1.0049375295639038],
 'base.1.weight': [0.0, 1.0062828063964844],
 'base.4.0.bn1.bias': [-1.0043219327926636, 0.0, 1.000535011291504],
 'base.4.0.bn1.weight': [0.0, 1.0029528141021729],
 'base.4.0.bn2.bias': [-1.0083696842193604, 0.0, 1.0035288333892822],
 'base.4.0.bn2.weight': [0.0, 0.9989628195762634],
 'base.4.0.conv1.weight': [-0.9992982745170593, 0.0, 0.9992982149124146],
 'base.4.0.conv2.weight': [-0.9979877471923828, 0.0, 0.9979877471923828],
 'base.4.1.bn1.bias': [-1.0035675764083862, 0.0, 1.0016995668411255],
 'base.4.1.bn1.weight': [0.0, 1.001746654510498],
 'base.4.1.bn2.bias': [-1.008662223815918, 0.0, 0.998275637626648],
 'base.4.1.bn2.weight': [0.0, 0.9972245097160339],
 'base.4.1.conv1.weight': [-0.9981896281242371, 0.0, 0.9981896281242371],
 'base.4.1.conv2.weight': [-1.0003037452697754, 0.0, 1.0003037452697754],
 'base.5.0.bn1.bias': [-1.0056942701339722, 0.0, 1.0

In [27]:
class PseudoData(Dataset):
    
    def __init__(self, data, label):
        
        self.data = data
        self.label = label

    def __getitem__(self, index):
        
        return self.data[index], self.label[index]

    def __len__(self):
        return len(self.data)

corr_label = torch.tensor(corr_label).type(torch.FloatTensor).to('cuda')
pseudo_dataLoader = DataLoader(dataset = PseudoData(corr_images, corr_label), batch_size = 1000, shuffle=False) 

In [28]:
def getResult(test_example, test_label, model, st):
    model.load_state_dict(st)
    model.eval()
    with torch.no_grad():
        output_ = model(test_example.float()) 
        pred = output_.data.max(1, keepdim=True)[1]
        z = pred.eq(test_label.data.view_as(pred)).flatten()
        return torch.where(z == False)[0]

In [29]:
def populateResults(idx, batch_id, old_state, new_state, w_images, n_ic, n_oc, row, column, weight_name):
  key = 'img_id'+str(idx)+'_batch_'+str(batch_id)
  if key not in w_images:
    w_images[key] = {}
    w_images[key]['location'] = []
    w_images[key]['location'].append((weight_name, n_ic, n_oc, row, column, old_state, new_state))
    w_images[key]['weight_states'] = []
    w_images[key]['weight_states'].append(weight_name + ' : ' + str(old_state) + ' --> ' + str(new_state))

  else:
    w_images[key]['location'].append((weight_name, n_ic, n_oc, row, column, old_state, new_state))
    w_images[key]['weight_states'].append(weight_name + ' : ' + str(old_state) + ' --> ' + str(new_state))

In [30]:
def populateResults1(idx, batch_id, old_state, new_state, w_images, row, column, weight_name):
  key = 'img_id'+str(idx)+'_batch_'+str(batch_id)
  if key not in w_images:
    w_images[key] = {}
    w_images[key]['location'] = []
    w_images[key]['location'].append((weight_name, row, column, old_state, new_state))
    w_images[key]['weight_states'] = []
    w_images[key]['weight_states'].append(weight_name + ' : ' + str(old_state) + ' --> ' + str(new_state))

  else:
    w_images[key]['location'].append((weight_name, row, column, old_state, new_state))
    w_images[key]['weight_states'].append(weight_name + ' : ' + str(old_state) + ' --> ' + str(new_state))

In [31]:
# For bias weight mutation

def bias_mutation(st, superSet, w_images, img,label, batch_id, weight_name):

  if ('pos' in pos_neg_layerwise[weight_name]) and ('neg' in pos_neg_layerwise[weight_name]):
    un = pos_neg_layerwise[weight_name]['pos'], pos_neg_layerwise[weight_name]['neg']

    var_neg1 = torch.where(st==un[1])[0]
    var_1 = torch.where(st==un[0])[0]

  else:
    try:
      un = pos_neg_layerwise[weight_name]['pos']

      var_neg1 = torch.where(st==un*1000)[0]
      var_1 = torch.where(st==un)[0]

    except KeyError:
      un = pos_neg_layerwise[weight_name]['neg']

      var_neg1 = torch.where(st==un)[0]
      var_1 = torch.where(st==un*1000)[0]


  if (var_neg1.nelement() == 0.) and (var_1.nelement() == 0.):
    pass
  else:
    for column in torch.cat((var_neg1, var_1), 0):
      row = 0
      column = column.item()

      org_val_real = st[column].item() # For bias part

      if type(un) == tuple:
          pos, neg = un   # Positive, Negative 
          if (org_val_real < 0.) and (org_val_real == neg):
            org_val = neg

            old_state = neg
            new_state = 0.
            superSet.add((weight_name, row, column, -1, 0))

            st[column] = new_state

            """getResult() will tell whether the passed test example is classified to other class
            or not after mutation, if it is classified to other class, then it returns True else False and 
            populateResults() will maintain the record of test examples which has been misclassified
            like location(row(which neuron), column(indices)), old_state(old_value of weight element), 
            new_state (mutated value of weight element), images"""

            
            [populateResults1(idx.item(), batch_id, -1, 0, w_images, row, column, weight_name) \
             for idx in getResult(img, label, model, state_dict)]
          
            new_state = pos
            superSet.add((weight_name, row, column, -1, 1))

            st[column] = new_state

            [populateResults1(idx.item(), batch_id, -1, 1, w_images, row, column, weight_name) \
             for idx in getResult(img, label, model, state_dict)]
            
            st[column] = org_val

          elif (org_val_real > 0.) and (org_val_real == pos):

            org_val = pos
            old_state = pos
            new_state = 0.
            superSet.add((weight_name, row, column, 1, 0))
            st[column] = new_state

            [populateResults1(idx.item(), batch_id, 1, 0, w_images, row, column, weight_name) \
             for idx in getResult(img, label, model, state_dict)]
          
            new_state = neg
            superSet.add((weight_name, row, column, 1, -1))

            st[column] = new_state
            
            [populateResults1(idx.item(), batch_id, 1, -1, w_images, row, column, weight_name) \
             for idx in getResult(img, label, model, state_dict)]
            
            st[column] = org_val


      
      ############# Negative ################################
      elif (un < 0) and (org_val_real == un):
          org_val = un
          old_state = un
          new_state = 0.
          superSet.add((weight_name, row, column, -1, 0))
          st[column] = new_state
            
          [populateResults1(idx.item(), batch_id, -1, 0, w_images, row, column, weight_name) \
           for idx in getResult(img, label, model, state_dict)]
          
          st[column] = org_val


      elif (un > 0) and (org_val_real == un):  ############# Positive  ################################
          org_val = un
          old_state = un
          new_state = 0.
          superSet.add((weight_name, row, column, 1, 0))
          st[column] = new_state
          [populateResults1(idx.item(), batch_id, 1, 0, w_images, row, column, weight_name) \
           for idx in getResult(img, label, model, state_dict)]

          st[column] = org_val


      #################################################################################################################

In [32]:
# Saving length of shape for each layer
layers_shape = {}

for i in layer_name:
  layers_shape[i] = len(state_dict[i].shape)

In [33]:
print(layers_shape)

{'base.0.weight': 4, 'base.1.bias': 1, 'base.1.weight': 1, 'base.4.0.conv1.weight': 4, 'base.4.0.bn1.bias': 1, 'base.4.0.bn1.weight': 1, 'base.4.0.conv2.weight': 4, 'base.4.0.bn2.bias': 1, 'base.4.0.bn2.weight': 1, 'base.4.1.conv1.weight': 4, 'base.4.1.bn1.bias': 1, 'base.4.1.bn1.weight': 1, 'base.4.1.conv2.weight': 4, 'base.4.1.bn2.bias': 1, 'base.4.1.bn2.weight': 1, 'base.5.0.conv1.weight': 4, 'base.5.0.bn1.bias': 1, 'base.5.0.bn1.weight': 1, 'base.5.0.conv2.weight': 4, 'base.5.0.bn2.bias': 1, 'base.5.0.bn2.weight': 1, 'base.5.0.downsample.0.weight': 4, 'base.5.0.downsample.1.bias': 1, 'base.5.0.downsample.1.weight': 1, 'base.5.1.conv1.weight': 4, 'base.5.1.bn1.bias': 1, 'base.5.1.bn1.weight': 1, 'base.5.1.conv2.weight': 4, 'base.5.1.bn2.bias': 1, 'base.5.1.bn2.weight': 1, 'base.6.0.conv1.weight': 4, 'base.6.0.bn1.bias': 1, 'base.6.0.bn1.weight': 1, 'base.6.0.conv2.weight': 4, 'base.6.0.bn2.bias': 1, 'base.6.0.bn2.weight': 1, 'base.6.0.downsample.0.weight': 4, 'base.6.0.downsample.1.

In [34]:
# Injecting fault by mutating the ternary weight and check whether it is classified to other class than before by the model. 

fetchimg_totImg_coverage_totCov = []

w_images = {}
superSet = set()


batch_id = 0
for img,label  in tqdm(pseudo_dataLoader):

    for weight_name in layer_name:

        # Selecting hidden layer to iterate its weight elements
        st = state_dict[weight_name]


        if ('bias' in weight_name) or (layers_shape[weight_name] == 1):
          bias_mutation(st, superSet, w_images, img,label, batch_id, weight_name)
          continue

        if ('conv' in weight_name) or (layers_shape[weight_name] == 4):

          ic, oc, k,k1 = st.shape # ic = in_channel, oc= out_channel, k and k1 is the number of rows and columns of a kernel

          for n_ic in range(ic):
            for n_oc in range(oc):
              kt =  st[n_ic][n_oc] # Fetching Kernel matrix or tensor

              # Iterating number of neurons for the selected hidden layer
              for row in range(k):

                # Fetching indices for -w and w weight value for the selected kernel
                """Since each kernel has weight vector, so by fetching indices
                we will come to know, which indices of the kernel contains weights -w or w weight"""

                ###########################################################################################

                if ('pos' in pos_neg_layerwise[weight_name]) and ('neg' in pos_neg_layerwise[weight_name]):
                  un = pos_neg_layerwise[weight_name]['pos'], pos_neg_layerwise[weight_name]['neg']

                  var_neg1 = torch.where(kt[row]==un[1])[0]
                  var_1 = torch.where(kt[row]==un[0])[0]

                else:
                  try:
                    un = pos_neg_layerwise[weight_name]['pos']

                    var_neg1 = torch.where(kt[row]==un*1000)[0]
                    var_1 = torch.where(kt[row]==un)[0]

                  except KeyError:
                    un = pos_neg_layerwise[weight_name]['neg']

                    var_neg1 = torch.where(kt[row]==un)[0]
                    var_1 = torch.where(kt[row]==un*1000)[0]

                ##########################################################################################

                if (var_neg1.nelement() == 0.) and (var_1.nelement() == 0.):
                  continue
                else:
                  for column in torch.cat((var_neg1, var_1), 0):

                    column = column.item()

                    org_val_real = kt[row][column].item()

                    ##################################################################################################################

                    if type(un) == tuple:
                      pos, neg = un  # Positive, Negative

                      if (org_val_real < 0.) and (org_val_real == neg):
                        org_val = neg
                        old_state = neg
                        new_state = 0.
                        superSet.add((weight_name, n_ic, n_oc, row, column, -1, 0))

                        st[n_ic][n_oc][row][column] = new_state
                        
                        """getResult() will tell whether the passed test example is classified to other class
                        or not after mutation, if it is classified to other class, then it returns True else False and 
                        populateResults() will maintain the record of test examples which has been misclassified
                        like location(row(which neuron), column(indices)), old_state(old_value of weight element), 
                        new_state (mutated value of weight element), images"""

                        
                        [populateResults(idx.item(), batch_id, -1, 0, w_images, n_ic, n_oc, row, column, weight_name) \
                         for idx in getResult(img, label, model, state_dict)]
                        
                        new_state = pos
                        superSet.add((weight_name, n_ic, n_oc, row, column, -1, 1))
                        st[n_ic][n_oc][row][column] = new_state

                            
                        [populateResults(idx.item(), batch_id, -1, 1, w_images, n_ic, n_oc, row, column, weight_name) \
                         for idx in getResult(img, label, model, state_dict)]
                        st[n_ic][n_oc][row][column] = org_val # Setting back to original value

                    
                      elif (org_val_real > 0.) and (org_val_real == pos):
                        org_val = pos
                        old_state = pos
                        new_state = 0.
                        superSet.add((weight_name, n_ic, n_oc, row, column, 1, 0))

                        st[n_ic][n_oc][row][column] = new_state
                        
                        [populateResults(idx.item(), batch_id, 1, 0, w_images, n_ic, n_oc, row, column, weight_name) \
                         for idx in getResult(img, label, model, state_dict)]
                        
                        new_state = neg
                        superSet.add((weight_name, n_ic, n_oc, row, column, 1, -1))
                        st[n_ic][n_oc][row][column] = new_state

                            
                        [populateResults(idx.item(), batch_id, 1, -1, w_images, n_ic, n_oc, row, column, weight_name) \
                         for idx in getResult(img, label, model, state_dict)]
                        
                        st[n_ic][n_oc][row][column] = org_val # Setting back to original value




                    elif (un < 0) and (org_val_real == un): ########################## Negative ###########################################
                      org_val = un
                      old_state = un
                      new_state = 0.
                      superSet.add((weight_name,  n_ic, n_oc, row, column, -1, 0))
                      st[n_ic][n_oc][row][column] = new_state
                        
                      [populateResults(idx.item(), batch_id, -1, 0, w_images, n_ic, n_oc, row, column, weight_name) \
                       for idx in getResult(img, label, model, state_dict)]
                      
                      st[row][column] = org_val


                    elif (un > 0) and (org_val_real == un): ########################## Positive ###########################################
                      org_val = un
                      old_state = un
                      new_state = 0.
                      superSet.add((weight_name, n_ic, n_oc, row, column, 1, 0))
                      st[n_ic][n_oc][row][column] = new_state
                        
                      [populateResults(idx.item(), batch_id, 1, 0, w_images, n_ic, n_oc, row, column, weight_name) \
                       for idx in getResult(img, label, model, state_dict)]
                      st[row][column] = org_val


    
            
        
        # For fully connected layers
        elif layers_shape[weight_name] == 2:
          rows = state_dict[weight_name].shape[0]

          # Iterating number of neurons for the selected hidden layer
          for row in range(rows):

            # Fetching indices for -w and w weight value for the selected neuron
            """Since each neuron has weight vector, so by fetching indices
            we will come to know, which indices of the neuron contains weights -w or w weight"""

            #########################################################################################

            if ('pos' in pos_neg_layerwise[weight_name]) and ('neg' in pos_neg_layerwise[weight_name]):
              un = pos_neg_layerwise[weight_name]['pos'], pos_neg_layerwise[weight_name]['neg']

              var_neg1 = torch.where(st[row]==un[1])[0]
              var_1 = torch.where(st[row]==un[0])[0]

            else:
              try:
                un = pos_neg_layerwise[weight_name]['pos']

                var_neg1 = torch.where(st[row]==un*1000)[0]
                var_1 = torch.where(st[row]==un)[0]

              except KeyError:
                un = pos_neg_layerwise[weight_name]['neg']

                var_neg1 = torch.where(st[row]==un)[0]
                var_1 = torch.where(st[row]==un*1000)[0]


            #########################################################################################


            if (var_neg1.nelement() == 0.) and (var_1.nelement() == 0.):
              continue
            else:
              for column in torch.cat((var_neg1, var_1), 0):

                column = column.item()

                org_val_v1 = st[row][column].item()


                #########################################################################################

                if type(un) == tuple:
                  pos, neg = un  # Positive, Negative

                  if (org_val_v1 < 0.) and (neg == org_val_v1):

                    org_val = neg
                    old_state = neg
                    new_state = 0.
                    superSet.add((weight_name, row, column, -1, 0))

                    st[row][column] = new_state
                    
                    [populateResults1(idx.item(), batch_id, -1, 0, w_images, row, column, weight_name) \
                     for idx in getResult(img, label, model, state_dict)]
                    
                    new_state = pos
                    superSet.add((weight_name, row, column, -1, 1))
                    st[row][column] = new_state
                        
                    [populateResults1(idx.item(), batch_id, -1, 1, w_images, row, column, weight_name) \
                     for idx in getResult(img, label, model, state_dict)]
                    st[row][column] = org_val


                  elif (org_val_v1 > 0.) and (pos == org_val_v1):

                    org_val = pos
                    old_state = pos
                    new_state = 0.
                    superSet.add((weight_name, row, column, 1, 0))

                    st[row][column] = new_state
                    
                    [populateResults1(idx.item(), batch_id, 1, 0, w_images, row, column, weight_name) \
                     for idx in getResult(img, label, model, state_dict)]
                    
                    new_state = neg
                    superSet.add((weight_name, row, column, 1, -1))
                    st[row][column] = new_state
                        
                    [populateResults1(idx.item(), batch_id, 1, -1, w_images, row, column, weight_name) \
                     for idx in getResult(img, label, model, state_dict)]
                    st[row][column] = org_val


 
                elif (un < 0) and (un == org_val_v1):  ################# Negative ###############################################################
                  org_val = un  
                  old_state = un
                  new_state = 0.
                  superSet.add((weight_name, row, column, -1, 0))
                  st[row][column] = new_state
                    
                  [populateResults1(idx.item(), batch_id, -1, 0, w_images, row, column, weight_name) \
                   for idx in getResult(img, label, model, state_dict)]

                  st[row][column] = org_val


                elif (un > 0) and (un == org_val_v1):  ################# Positive ###############################################################
                  org_val = un
                  old_state = un
                  new_state = 0.
                  superSet.add((weight_name, row, column, 1, 0))
                  st[row][column] = new_state
                    
                  [populateResults1(idx.item(), batch_id, 1, 0, w_images, row, column, weight_name) \
                   for idx in getResult(img, label, model, state_dict)]
                  
                  st[row][column] = org_val


    batch_id += 1


    # Fetching locations and total number of images
    ################################################

    print(f"Total number of elements in superSet is : {len(superSet)}.",'\n')

    # joblib.dump(w_images, open('w_images_'+str(batch_id)+'.pkl','wb')) 

    image_locCounts = [] # (image_id,locationCounts)

    for img in w_images.keys():
        locCounts = len(w_images[img]['location'])
        image_locCounts.append((locCounts,img))
            
    image_locCounts.sort(reverse=True)


    netSetOfImages = []
    progressingSet = []
    max_locCounts_img = image_locCounts[0][1] # An image that has encompassed maximum locations.

    netSetOfImages.append(max_locCounts_img)
    progressingSet += set(w_images[max_locCounts_img]['location'])


    for i in range(1,len(image_locCounts)):
        img = image_locCounts[i][1]
        locs = w_images[img]['location']

        if set(locs).issubset(progressingSet):
            pass
        else:
            progressingSet += list(set(locs) - set(progressingSet))
            netSetOfImages.append(img)


    if set(superSet).issubset(progressingSet):
        print("All match done!")
    else:
        print("There are {} locations that did not match.".format(len(set(superSet) - set(progressingSet))))

    print()
    print("Total number of locations {} are encompassed out of {}.".format(len(set(progressingSet)), len(superSet)))


    print("Total number of elements in a weight matrix is ", len(superSet))

    print("Total number of images fetched ",len(netSetOfImages),'\n')

    fetchimg_totImg_coverage_totCov.append(( len(netSetOfImages), len(set(progressingSet)), len(superSet) ))

    # joblib.dump(fetchimg_totImg_coverage_totCov, open('res'+str(numImgs)+'.pkl','wb')) 

    # Superset 
    # joblib.dump(superSet, open('superSet.pkl','wb'))    

                
      
    print("Output \n")
    print(fetchimg_totImg_coverage_totCov)

    #joblib.dump(fetchimg_totImg_coverage_totCov, open('op_128.pkl', 'wb'))


  0%|          | 0/10 [00:00<?, ?it/s]

Total number of elements in superSet is : 161639. 



 10%|█         | 1/10 [1:19:37<11:56:36, 4777.36s/it]

There are 111117 locations that did not match.

Total number of locations 50522 are encompassed out of 161639.
Total number of elements in a weight matrix is  161639
Total number of images fetched  86 

Output 

[(86, 50522, 161639)]
Total number of elements in superSet is : 161639. 



 20%|██        | 2/10 [2:39:29<10:38:07, 4785.94s/it]

...

There are 83031 locations that did not match.

Total number of locations 78608 are encompassed out of 161639.
Total number of elements in a weight matrix is  161639
Total number of images fetched  465 

Output 

[(86, 50522, 161639), (136, 61476, 161639), (193, 66115, 161639), (303, 70365, 161639), (368, 72985, 161639), (411, 74435, 161639), (431, 75137, 161639), (465, 78608, 161639)]
Total number of elements in superSet is : 161639. 



 90%|█████████ | 9/10 [12:06:08<1:21:15, 4875.23s/it]

There are 77156 locations that did not match.

Total number of locations 84483 are encompassed out of 161639.
Total number of elements in a weight matrix is  161639
Total number of images fetched  479 

Output 

[(86, 50522, 161639), (136, 61476, 161639), (193, 66115, 161639), (303, 70365, 161639), (368, 72985, 161639), (411, 74435, 161639), (431, 75137, 161639), (465, 78608, 161639), (479, 84483, 161639)]
Total number of elements in superSet is : 161639. 



100%|██████████| 10/10 [13:09:58<00:00, 4739.85s/it]

There are 76608 locations that did not match.

Total number of locations 85031 are encompassed out of 161639.
Total number of elements in a weight matrix is  161639
Total number of images fetched  512 

Output 

[(86, 50522, 161639), (136, 61476, 161639), (193, 66115, 161639), (303, 70365, 161639), (368, 72985, 161639), (411, 74435, 161639), (431, 75137, 161639), (465, 78608, 161639), (479, 84483, 161639), (512, 85031, 161639)]


In [35]:
# Fault coverage
fetchimg_totImg_coverage_totCov[-1][1]/fetchimg_totImg_coverage_totCov[-1][2]

0.5260549743564362

In [ ]:
# We also have to consider dead neurons in the model that we will get from another notebook. Then we will have total fault coverage. 